# Portfolio Analysis

This is just out of curiosity and just for fun, i.e., **this is not financial advice**.

In this notebook, I compute a **Markowitz portfolio optimization** for three selected cryptocurrencies: BTC, ETH, XMR.
I chose those due to their value proposition, explained in the previous notebook `01_BitcoinAnalysis`:
- BTC: alternative open currency
- ETH: general blockchain
- XMR: alternative private currency

The Markowitz portfolio optimization consists in performing a **Monte Carlo Simulation** in the portfolio allocation space (i.e., the ratio assigned to each of the selected assets) to obtain the return and volatility (variance) values of each simulated sample.
The best allocation is the one with the highest Sharpe ratio, which is basically the risk adjusted return:

$S = \frac{R_p - R_f}{\sigma_p}$, where

- $R_p$: expected portfolio return
- $R_f$: risk-free return: return we would have had if we had put our money in an investment such as savings account, EURIBOR, or treasury bonds that are considered to be risk free. In the EU, the EURIBOR is currently (Dec. 2021) -0.5%, i.e., one bank **pays** a fee of -0.5% when it lends money to another one
- $\sigma_p$: portfolio standard deviation

So, the higher $S$ the better, because we'd have high returns with low volatility!

Note that William Sharpe (Nobel laureate) originally presented the formula for **yearly returns**: *mean* daily returns and std. dev. of the whole year. However, we deal often with daily, weekly and monthly sampling series, thus the **Anualized Sharpe Ratio** is obtained by multiplying the Sharpe ratio with the following k constant:

`ASR = k * S`

k = 
- `sqrt(252)` for daily sampling series (stocks, since traded during working days)
- `sqrt(365)` for daily sampling series (cryptos, since traded daily) -- or that's what I understand
- `sqrt(52)` for weekly sampling series
- `sqrt(12)` for monthly sampling series

Typical interpretation of the Annualized Sharpe Ratio (ASR) values:
- 1: go-to for investors
- 2: very good
- 3: excellent

Note that I learned most of the concepts used here from José Portilla's videos in [Python for Financial Analysis and Algorithmic Trading](https://www.udemy.com/course/python-for-finance-and-trading-algorithms/) and Robert Shiller's Coursera course [Financial Markets](https://www.coursera.org/learn/financial-markets-global).

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [5]:
# Load and prepare datasets
df_btc = pd.read_csv('data/Gemini_BTCUSD_2015_2021_hourly.csv',header=0)
df_btc['Date'] = pd.to_datetime(df_btc['Date'])
df_btc.set_index('Date',inplace=True)
df_eth = pd.read_csv('data/Bitstamp_ETHUSD_2017_2021_hourly.csv',header=0)
df_eth['Date'] = pd.to_datetime(df_eth['Date'])
df_eth.set_index('Date',inplace=True)
df_xmr = pd.read_csv('data/Binance_XMRUSD_2020_2021_hourly.csv',header=0)
df_xmr['Date'] = pd.to_datetime(df_xmr['Date'])
df_xmr.set_index('Date',inplace=True)

In [6]:
df_btc.head()

,Open,High,Low,Close,Volume BTC
Date,,,,,
2015-10-08 13:00:00,0.00,245.00,0.00,245.00,0.606654
2015-10-08 14:00:00,245.00,245.00,244.50,245.00,4.453649
2015-10-08 15:00:00,245.00,245.00,244.92,244.92,3.016926
2015-10-08 16:00:00,244.92,244.92,244.25,244.25,3.895252
2015-10-08 17:00:00,244.25,244.99,244.02,244.99,3.920632


In [7]:
df_eth.head()

,Open,High,Low,Close,Volume ETH,Volume USD
Date,,,,,,
2017-08-16 16:00:00,300.0,300.0,300.0,300.0,0.02,6.0
2017-08-16 17:00:00,300.0,300.0,300.0,300.0,0.00,0.0
2017-08-16 18:00:00,300.0,300.0,300.0,300.0,0.00,0.0
2017-08-16 19:00:00,300.0,300.0,300.0,300.0,0.00,0.0
2017-08-16 20:00:00,300.0,300.0,300.0,300.0,0.00,0.0


In [8]:
df_xmr.head()

,Open,High,Low,Close,Volume XMR,Volume USDT
Date,,,,,,
2020-02-03 08:00:00,75.81,77.21,75.70,76.66,9094.826,6.973109e+05
2020-02-03 09:00:00,76.66,76.74,75.98,76.12,10198.387,7.793740e+05
2020-02-03 10:00:00,76.12,76.60,75.92,76.50,14820.044,1.128870e+06
2020-02-03 11:00:00,76.50,78.27,76.33,77.44,18219.038,1.409690e+06
2020-02-03 12:00:00,77.44,77.99,77.35,77.39,13923.253,1.081595e+06


In [11]:
# Compute maximum possible rane in the data we have
date_start = max(df_btc.index[0], df_eth.index[0], df_xmr.index[0])
date_end = min(df_btc.index[-1], df_eth.index[-1], df_xmr.index[-1])

In [16]:
df = pd.DataFrame()
df["BTC"] = df_btc["Close"].loc[date_start:date_end]
df["ETH"] = df_eth["Close"].loc[date_start:date_end]
df["XMR"] = df_xmr["Close"].loc[date_start:date_end]

In [17]:
df.head()

,BTC,ETH,XMR
Date,,,
2020-02-03 08:00:00,9342.64,189.63,76.66
2020-02-03 09:00:00,9317.00,188.43,76.12
2020-02-03 10:00:00,9292.16,187.92,76.50
2020-02-03 11:00:00,9320.53,189.53,77.44
2020-02-03 12:00:00,9326.08,189.53,77.39


In [19]:
# Get number of datapoints
k = df.shape[0]